# Intialize NMRlipids databank

In [1]:
import os
import sys
import numpy as np
import json
import matplotlib.pyplot as plt
import MDAnalysis
import urllib.request
import yaml
import random


# This define the locate path for the NMRlipids databank on your computer. 
# Default is that this template repository and the NMRlipids databank repository are cloned to the same folder.
# If this is not the case, changed this to the folder where the NMRlipids databank repository is located.
databankPath =  '../../Databank/'

# This enales the access to functions defined in the NMRlipids databank.
sys.path.insert(1, databankPath + '/Scripts/BuildDatabank/')
from databankLibrary import * 

# This initializes the databank and stores the information of all simulations into a list.
# Each list item contains the information from README.yaml file of the given simulation.
systems = initialize_databank(databankPath)

# Select random simulation for example and show its content

In [2]:
 ### To exemplify the analysis of any simulation, we select randomly one simulation ID
randomID = random.randint(1, len(systems))
print('ID of the selected simulation: ', randomID)


 ### This loops over all the simulations in the databank
for system in systems:
    
    ### To lighten up the example, we randomly select one simulation
    ### To run analyses for all simulations, comment out two lines below
    if system['ID'] != randomID:
        continue
    
    ### This prints the information about the selected simulation
    print_README(system)
 

 ### Remove comments from below two lines to see explanation on content in each simulation
#print('\n Explanation of simulation content')
#print_README('example')

ID of the selected simulation:  45
DOI:
  10.5281/zenodo.3540513
SOFTWARE:
  gromacs
TRJ:
  [['DPPC_10_310_DCHOL.xtc']]
TPR:
  [['DPPC_10_310_DCHOL.tpr']]
PREEQTIME:
  0
TIMELEFTOUT:
  0
DIR_WRK:
  /usr/home/bort/Databank
PUBLICATION:
  None
AUTHORS_CONTACT:
  Javanainen, Matti; Martinez-Seara, Hector; Vattulainen, Ilpo
SYSTEM:
  1032DPPC_120DCHOL_46440SOL_124SOD_124CLA_310K
SOFTWARE_VERSION:
  None
FF:
  Slipids
FF_SOURCE:
  None
FF_DATE:
  pre-2020
CPT:
  [['DPPC_10_310_DCHOL.cpt']]
LOG:
  None
TOP:
  None
COMPOSITION:
  {'DPPC': {'NAME': 'DPPC', 'MAPPING': 'mappingDPPCslipid.yaml', 'COUNT': [516, 516]}, 'DCHOL': {'NAME': 'DCHL', 'MAPPING': 'mappingDCHOLslipids.yaml', 'COUNT': [60, 60]}, 'SOL': {'NAME': 'SOL', 'MAPPING': 'mappingTIP3PwaterSlipids.yaml', 'COUNT': 46440}, 'SOD': {'NAME': 'NA', 'MAPPING': 'mappingNA.yaml', 'COUNT': 124}, 'CLA': {'NAME': 'CL', 'MAPPING': 'mappingCL.yaml', 'COUNT': 124}}
TRAJECTORY_SIZE:
  7370876156
TRJLENGTH:
  700100.0
TEMPERATURE:
  310.0
NUMBER_OF_AT

# Show analyzed properties stored in the NMRlipids databank for the selected system: 
## membrane area per lipid, thickness, relative equilibration time, C-H bond order parameters and X-ray scattering form factors

In [5]:
for system in systems:
    
    ### To lighten up the example, we randomly select one simulation
    ### To run analyses for all simulations, comment out two lines below
    if system['ID'] != randomID:
        continue
    
    APL = CalcAreaPerMolecule(system)
    print('Area per lipid:' , APL)


Area per lipid: 52.588789253832466
